# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0cb4b0c9d0>
├── 'a' --> tensor([[-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746]])
└── 'x' --> <FastTreeValue 0x7f0cb4a98b20>
    └── 'c' --> tensor([[ 1.2206, -1.2723, -1.0659,  0.1906],
                        [-1.0474,  1.3397, -1.4550, -0.9992],
                        [-0.0139, -0.5839,  0.6985,  0.0647]])

In [4]:
t.a

tensor([[-0.8762, -0.9651,  0.1883],
        [ 2.0656, -0.4280,  1.7746]])

In [5]:
%timeit t.a

80.3 ns ± 0.723 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0cb4b0c9d0>
├── 'a' --> tensor([[ 1.1886,  0.6863,  0.3304],
│                   [-0.3612,  0.0446,  1.3160]])
└── 'x' --> <FastTreeValue 0x7f0cb4a98b20>
    └── 'c' --> tensor([[ 1.2206, -1.2723, -1.0659,  0.1906],
                        [-1.0474,  1.3397, -1.4550, -0.9992],
                        [-0.0139, -0.5839,  0.6985,  0.0647]])

In [7]:
%timeit t.a = new_value

79.7 ns ± 0.697 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8762, -0.9651,  0.1883],
               [ 2.0656, -0.4280,  1.7746]]),
    x: Batch(
           c: tensor([[ 1.2206, -1.2723, -1.0659,  0.1906],
                      [-1.0474,  1.3397, -1.4550, -0.9992],
                      [-0.0139, -0.5839,  0.6985,  0.0647]]),
       ),
)

In [10]:
b.a

tensor([[-0.8762, -0.9651,  0.1883],
        [ 2.0656, -0.4280,  1.7746]])

In [11]:
%timeit b.a

69.8 ns ± 0.847 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2205,  0.8103, -1.4005],
               [ 0.8096,  0.7091,  1.3942]]),
    x: Batch(
           c: tensor([[ 1.2206, -1.2723, -1.0659,  0.1906],
                      [-1.0474,  1.3397, -1.4550, -0.9992],
                      [-0.0139, -0.5839,  0.6985,  0.0647]]),
       ),
)

In [13]:
%timeit b.a = new_value

655 ns ± 5.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.12 µs ± 49.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15.1 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

307 µs ± 19.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

302 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0c097b64f0>
├── 'a' --> tensor([[[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]],
│           
│                   [[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]],
│           
│                   [[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]],
│           
│                   [[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]],
│           
│                   [[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]],
│           
│                   [[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]],
│           
│                   [[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]],
│           
│                   [[-0.8762, -0.9651,  0.1883],
│                    [ 2.0656, -0.4280,  1.7746]]])
└── 'x' --> <FastTreeValue 0x7f0cb4ad04c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.9 µs ± 4.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0c103b28e0>
├── 'a' --> tensor([[-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746],
│                   [-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746],
│                   [-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746],
│                   [-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746],
│                   [-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746],
│                   [-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746],
│                   [-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746],
│                   [-0.8762, -0.9651,  0.1883],
│                   [ 2.0656, -0.4280,  1.7746]])
└── 'x' --> <FastTreeValue 0x7f0c0983e490>
    └── 'c' --> tensor([[ 1.2206, -1.2723, -1.0659,  0.1906],
                        [-1.0474,  1.3397, -1.4550, -0.9992],
                 

In [23]:
%timeit t_cat(trees)

42.8 µs ± 779 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

84.5 µs ± 974 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2206, -1.2723, -1.0659,  0.1906],
                       [-1.0474,  1.3397, -1.4550, -0.9992],
                       [-0.0139, -0.5839,  0.6985,  0.0647]],
              
                      [[ 1.2206, -1.2723, -1.0659,  0.1906],
                       [-1.0474,  1.3397, -1.4550, -0.9992],
                       [-0.0139, -0.5839,  0.6985,  0.0647]],
              
                      [[ 1.2206, -1.2723, -1.0659,  0.1906],
                       [-1.0474,  1.3397, -1.4550, -0.9992],
                       [-0.0139, -0.5839,  0.6985,  0.0647]],
              
                      [[ 1.2206, -1.2723, -1.0659,  0.1906],
                       [-1.0474,  1.3397, -1.4550, -0.9992],
                       [-0.0139, -0.5839,  0.6985,  0.0647]],
              
                      [[ 1.2206, -1.2723, -1.0659,  0.1906],
                       [-1.0474,  1.3397, -1.4550, -0.9992],
                       [-0.0139, -0.5839,  0.6985,  0.0647]],

In [26]:
%timeit Batch.stack(batches)

112 µs ± 949 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2206, -1.2723, -1.0659,  0.1906],
                      [-1.0474,  1.3397, -1.4550, -0.9992],
                      [-0.0139, -0.5839,  0.6985,  0.0647],
                      [ 1.2206, -1.2723, -1.0659,  0.1906],
                      [-1.0474,  1.3397, -1.4550, -0.9992],
                      [-0.0139, -0.5839,  0.6985,  0.0647],
                      [ 1.2206, -1.2723, -1.0659,  0.1906],
                      [-1.0474,  1.3397, -1.4550, -0.9992],
                      [-0.0139, -0.5839,  0.6985,  0.0647],
                      [ 1.2206, -1.2723, -1.0659,  0.1906],
                      [-1.0474,  1.3397, -1.4550, -0.9992],
                      [-0.0139, -0.5839,  0.6985,  0.0647],
                      [ 1.2206, -1.2723, -1.0659,  0.1906],
                      [-1.0474,  1.3397, -1.4550, -0.9992],
                      [-0.0139, -0.5839,  0.6985,  0.0647],
                      [ 1.2206, -1.2723, -1.0659,  0.1906],
                   

In [28]:
%timeit Batch.cat(batches)

207 µs ± 5.59 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

622 µs ± 50.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
